<a href="https://colab.research.google.com/github/SeoJiWon1/Mask-detection/blob/main/Webcam_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mask Detect with Webcam

In [ ]:
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
from sklearn.model_selection import train_test_split
import shutil
import copy
from PIL import Image
import glob

## Face & Mask Detector Preparation

In [ ]:
# Face Detector Path
face_dnn_model_name = 'C:/Users/user/Downloads/res10_300x300_ssd_iter_140000.caffemodel'
prototxt_name = 'C:/Users/user/Downloads/deploy.prototxt.txt'

# Mask Detector Path
# file_path = 'C:/Users/user/Downloads/mask_detector_resnet101.pth'
file_path = 'C:/Users/user/Downloads/mask_detector_mobilenet.pth'

In [ ]:
# 모델 객체 생성
face_detector = cv2.dnn.readNetFromCaffe(prototxt_name, face_dnn_model_name)
mask_detector = torch.load(file_path)

In [ ]:
# 변수 설정
# 얼굴 인식 최소 신뢰도
min_confidence = 0.3

# 이미지 크기
frame_width = 700

# Label
class_names = ['with_mask', 'without_mask']

In [ ]:
# 이미지 전처리 메서드
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model
#     pil_image = Image.open(image)
    pil_image = image
   
    image_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    img = image_transforms(pil_image)
    return img

## Mask Detection Method

In [ ]:
# Mask Detection 메서드
def classify_mask(image):
    device = torch.device("cpu")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(image)
    image = process_image(img)
    print('image_processed')
    img = image.unsqueeze_(0)
    img = image.float()

    mask_detector.eval()
    mask_detector.cpu()
    output = mask_detector(image)
    print(output,'##############output###########')
    check, predicted = torch.max(output, 1)
    print(predicted.data[0],"predicted")


    classification = predicted.data[0]
    index = int(classification)
    print(class_names[index])
    return class_names[index]

## Detect And Display Method

In [ ]:
# 이미지를 받아서 얼굴 인식 및 마스크 탐지 진행 후 반환 처리하는 메서드
def detectAndDisplay(frame):
    # frame_width 에 맞춰서 image resize
    # height, width = frame.shape[:2]
    (height, width) = frame.shape[:2]
    ratio = frame_width / width
    dimension = (frame_width, int(height * ratio))
    frame = cv2.resize(frame, dimension, interpolation = cv2.INTER_AREA)

    # 이미지를 300 x 300으로 size 조정하고 blob 만들기
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), scalefactor=1.0,
                                 size=(300, 300), mean=(104.0, 177.0, 123.0))    
    
    # blob 모델에 넣고 얼굴 detection 수행
    face_detector.setInput(blob)
    face_detections = face_detector.forward()

    result_frame = frame.copy()

    # Detections 한 수만큼 루프 돌기
    for i in range(0, face_detections.shape[2]):
        confidence = face_detections[0, 0, i, 2] # confidence는 detection한 확률

        # min_confidence 보다 큰 경우에만 detection으로 인정함
        if confidence > min_confidence:
            # box 좌표 구하기
            # box = face_detections[0, 0, i, 3:7] * np.array([frame_width, height, frame_width, height])
            box = face_detections[0, 0, i, 3:7] * np.array([frame_width, int(height * ratio), frame_width, int(height * ratio)])
            (x1, y1, x2, y2) = box.astype('int')
            
            # 얼굴 이미지 자르기
            face_input = frame[y1:y2, x1:x2]

            # 얼굴 이미지를 모델에 넣기
            label = classify_mask(face_input)
            print(f'Label : {label}')
            
            # 마스크 인식 결과 비교
            if label == 'with_mask':
                color = (0, 255, 0)
            else:
                color = (0, 0, 255)

            # 얼굴에 사각형 그리기
            cv2.rectangle(result_frame, pt1=(x1, y1), pt2=(x2, y2), thickness=2, color=color, lineType=cv2.LINE_AA)
            cv2.putText(result_frame, text=label, org=(x1, y1 - 10), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.8, color=color, thickness=2, lineType=cv2.LINE_AA)

    # 결과 출력
    cv2.imshow('frame', result_frame)

## Webcam Input & Output

In [ ]:
# Webcam 입출력
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, frame_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_width)

if cap.isOpened():
    while True:
        ret, frame = cap.read()
        if ret:
            detectAndDisplay(frame)
            if cv2.waitKey(1) != -1:
                break
        
cap.release()
cv2.destroyAllWindows()

## Face Tracking
- Tracking이 Detection보다 자원 소모가 적고 처리속도가 빠르다
- 처음에만 Detection하고 이후에는 Tracking을 해서 부드럽게 추적하도록 구현
- 시도를 해보고 있지만 안될수 있음..

In [ ]:
cv2.__version__

'4.6.0'

In [ ]:
# Object Tracker Mode 설정
tracker = cv2.legacy.TrackerKCF_create()

In [ ]:
# 변수 초기화
detected = False
frame_mode = 'Tracking'

In [ ]:
# 카메라 동영상 로드
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, frame_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_width)

# 원본 동영상 오픈 여부 확인
if not cap.isOpened:
    print('--(!) Error opening video capture')
    exit(0)

# trakers 객체 생성
# OvenCV 4.6.0 이후는 legacy에 들어가있음!! 버전 확인
trackers = cv2.legacy.MultiTracker_create()

if cap.isOpened():
    while True:
        ret, frame = cap.read()
        
        (height, width) = frame.shape[:2]
        ratio = frame_width / width
        dimension = (frame_width, int(height * ratio))
        frame = cv2.resize(frame, dimension, interpolation = cv2.INTER_AREA)
        
        if ret:
            # 얼굴인식-추적, 객체 탐지
            if detected:
                frame_mode = 'Tracking'
                ret, boxes = trackers.update(frame)
                
                for box in boxes:
                    (x1, y1, x2, y2) = [int(v) for v in box]
                
                # 얼굴 이미지 자르기
                face_input = frame[y1:y2, x1:x2]
                
                # 얼굴 이미지를 모델에 넣기
                label = classify_mask(face_input)
                print(f'Label : {label}')
                
                # 마스크 인식 결과 비교
                if label == 'with_mask':
                    color = (0, 255, 0)
                    label = 'With Mask %d%%' % (confidence * 100)
                else:
                    color = (0, 0, 255)
                    label = 'Without Mask %d%%' % (confidence * 100)

                # 얼굴에 사각형 그리기
                cv2.rectangle(frame, pt1=(x1, y1), pt2=(x2, y2), thickness=2, color=color, lineType=cv2.LINE_AA)
                cv2.putText(frame, text=label, org=(x1, y1 - 10), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.8, color=color, thickness=2, lineType=cv2.LINE_AA)
                cv2.putText(frame,text=str(frame_mode),org=(100,height-20), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1,color=(255,255,255),thickness=1,lineType=cv2.LINE_AA)
            
            else:
                frame_mode = 'Detection'

                blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), scalefactor=1.0,
                                     size=(300, 300), mean=(104.0, 177.0, 123.0))    
                
                # blob 모델에 넣고 얼굴 detection 수행
                face_detector.setInput(blob)
                face_detections = face_detector.forward()
                
                # Detections 한 수만큼 루프 돌기
                for i in range(0, face_detections.shape[2]):
                    confidence = face_detections[0, 0, i, 2] # confidence는 detection한 확률

                    # min_confidence 보다 큰 경우에만 detection으로 인정함
                    if confidence > min_confidence:
                        # box 좌표 구하기
                        box = face_detections[0, 0, i, 3:7] * np.array([frame_width, int(height * ratio), frame_width, int(height * ratio)])
                        (x1, y1, x2, y2) = box.astype('int')

                        # 얼굴 이미지 자르기
                        face_input = frame[y1:y2, x1:x2]

                        # 얼굴 이미지를 모델에 넣기
                        label = classify_mask(face_input)
                        print(f'Label : {label}')

                        # 마스크 인식 결과 비교
                        if label == 'with_mask':
                            color = (0, 255, 0)
                            label = 'With Mask %d%%' % (confidence * 100)
                        else:
                            color = (0, 0, 255)
                            label = 'Without Mask %d%%' % (confidence * 100)

                        # 얼굴에 사각형 그리기
                        cv2.rectangle(frame, pt1=(x1, y1), pt2=(x2, y2), thickness=2, color=color, lineType=cv2.LINE_AA)
                        cv2.putText(frame, text=label, org=(x1, y1 - 10), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.8, color=color, thickness=2, lineType=cv2.LINE_AA)
                        cv2.putText(frame,text=str(frame_mode),org=(100,height-20), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1,color=(255,255,255),thickness=1,lineType=cv2.LINE_AA)
                        
                # trackers 객체가 설정된 tracker mode로 boxing된 영역 tracking
                trackers.add(tracker, frame, (x1, y1, x2, y2))
                detected = True # detected mode 변경
            
            cv2.imshow('frame', frame)
            if cv2.waitKey(1) != -1:
                break
        
cap.release()
cv2.destroyAllWindows()